In [1]:
#https://www.kaggle.com/thebrownviking20/bert-multiclass-classification/notebook#BERT-implementation
mport numpy as np 
import pandas as pd
import re
import gc
import os
print(os.listdir("input"))
import fileinput
import string
import tensorflow as tf
import zipfile
import datetime
import sys

from nltk.tokenize import wordpunct_tokenize
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score, roc_auc_score

[]


In [7]:
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation
#from tensorflow.keras.layers.embeddings import Embedding
from sklearn.metrics import classification_report

In [8]:
valid_forms = ['am','are','were','was','is','been','being','be']
blank = '----'

In [9]:
def detect(tokens):
    return [t for t in tokens if t in valid_forms]
    
def replace_blank(tokens):
    return [blank if t in valid_forms else t for t in tokens]

def create_windows(tokens, window_size=3):
    X = []
    for i, word in enumerate(tokens):
        if word == blank:
            window = tokens[i-window_size:i] + tokens[i+1:i+window_size+1]
            window = ' '.join(window)
            X.append(window)    
    return X

In [129]:
f1 = open("internet_archive_scifi_v3.txt","r")
a = f1.read()
a = re.sub('[\n]', '', a)
tokens = wordpunct_tokenize(a)
y = detect(tokens)
tokens = replace_blank(tokens)
X = create_windows(tokens)
f1.close()

In [130]:
df = pd.DataFrame()
df["Text"] = X
df["Label"] = y

In [131]:
df.head()

,Text,Label
0,Publisher Editor IF published bi -,is
1,in this magazine fiction . Any,are
2,to actual persons coincidental . #,is
3,. The title selected after much,was
4,the theory it indicative of the,is


In [132]:
one_hot = pd.get_dummies(df["Label"])
df.drop(['Label'],axis=1,inplace=True)
df = pd.concat([df,one_hot],axis=1)
df.head()

,Text,am,are,be,been,being,is,was,were
0,Publisher Editor IF published bi -,0,0,0,0,0,1,0,0
1,in this magazine fiction . Any,0,1,0,0,0,0,0,0
2,to actual persons coincidental . #,0,0,0,0,0,1,0,0
3,. The title selected after much,0,0,0,0,0,0,1,0
4,the theory it indicative of the,0,0,0,0,0,1,0,0


In [133]:
df1 = df[:200000]

In [134]:
df1.shape

(200000, 9)

In [135]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df1["Text"].values, df1.drop(['Text'],axis=1).values, test_size=0.2, random_state=42)

In [137]:
X_train1 = X_train
X_test1 = X_test
y_train1 = y_train
y_test1 = y_test

In [138]:
#valid_forms = ['am','are','were','was','is','been','being','be']
X_test[4], y_test[4]

('field . Reports already coming in',
 array([0, 1, 0, 0, 0, 0, 0, 0], dtype=uint8))

In [139]:
vocabulary_size = 20000
tokenizer = Tokenizer(num_words= vocabulary_size)
tokenizer.fit_on_texts(X_train)
sequences = tokenizer.texts_to_sequences(X_train)
X_train = pad_sequences(sequences, maxlen=50)

In [140]:
sequences = tokenizer.texts_to_sequences(X_test)
X_test = pad_sequences(sequences, maxlen=50)

In [146]:
sequences[13]

[55, 13, 45]

In [152]:
X_test[13]

array([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 55, 13, 45])

In [153]:
X_test1[13]

'. " Where you going ?"'

In [179]:
X_test[13].shape

(50,)

In [183]:
sequences = tokenizer.texts_to_sequences(['how you today ?"'])
X_test_testing = pad_sequences(sequences, maxlen=50)
X_test_testing

array([[  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,  74,  13, 483]])

In [178]:
X_test_testing[0].shape

(50,)

In [25]:
from tensorflow.keras.layers import Embedding
model = Sequential()
model.add(Embedding(20000, 100, input_length=50))
model.add(Dropout(0.2))
model.add(Conv1D(64, 5, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(LSTM(100))
model.add(Dense(8, activation='softmax'))

W0917 13:12:36.869491 28340 deprecation.py:506] From C:\Users\gunawf01\AppData\Local\Continuum\anaconda3\envs\tfdeeplearning\lib\site-packages\tensorflow\python\keras\initializers.py:119: calling RandomUniform.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0917 13:12:37.007591 28340 deprecation.py:506] From C:\Users\gunawf01\AppData\Local\Continuum\anaconda3\envs\tfdeeplearning\lib\site-packages\tensorflow\python\ops\init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [26]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [27]:
model.fit(X_train, y_train,
                    batch_size=1024,
                    epochs=10,
                    verbose=1,
                    validation_split=0.1)

Train on 144000 samples, validate on 16000 samples


W0917 13:13:38.673088 28340 deprecation.py:323] From C:\Users\gunawf01\AppData\Local\Continuum\anaconda3\envs\tfdeeplearning\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/10
144000/144000 [==============================] - 320s 2ms/sample - loss: 1.3027 - acc: 0.5350 - val_loss: 0.8172 - val_acc: 0.7035
Epoch 2/10
144000/144000 [==============================] - 318s 2ms/sample - loss: 0.7240 - acc: 0.7316 - val_loss: 0.7145 - val_acc: 0.7297
Epoch 3/10
144000/144000 [==============================] - 261s 2ms/sample - loss: 0.6067 - acc: 0.7722 - val_loss: 0.6908 - val_acc: 0.7391
Epoch 4/10
144000/144000 [==============================] - 195s 1ms/sample - loss: 0.5295 - acc: 0.8027 - val_loss: 0.6980 - val_acc: 0.7392
Epoch 5/10
144000/144000 [==============================] - 194s 1ms/sample - loss: 0.4682 - acc: 0.8267 - val_loss: 0.7187 - val_acc: 0.7393
Epoch 6/10
144000/144000 [==============================] - 194s 1ms/sample - loss: 0.4171 - acc: 0.8460 - val_loss: 0.7479 - val_acc: 0.7344
Epoch 7/10
144000/144000 [==============================] - 205s 1ms/sample - loss: 0.3717 - acc: 0.8627 - val_loss: 0.7847 - val_acc: 0.7300
Epoch 

In [28]:
score = model.evaluate(X_test, y_test,
                       batch_size=256, verbose=1)
print('Test accuracy:', score[1])

40000/40000 [==============================] - 9s 221us/sample - loss: 0.9046 - acc: 0.7257
Test accuracy: 0.72565


In [29]:
preds = model.predict(X_test)
preds

array([[8.8679218e-01, 2.5310742e-03, 1.7630089e-03, ..., 1.0599068e-02,
        9.6763194e-02, 7.5688004e-04],
       [1.2780123e-07, 1.1998489e-07, 9.9985087e-01, ..., 1.4259963e-05,
        7.6977922e-05, 7.8345138e-06],
       [1.8735274e-04, 3.8915399e-01, 1.3187789e-05, ..., 1.1678348e-04,
        1.4086851e-03, 6.0909170e-01],
       ...,
       [4.8492208e-02, 2.0528250e-04, 4.0013092e-03, ..., 1.6399462e-02,
        9.0137970e-01, 8.7342842e-04],
       [1.1335555e-06, 5.6259778e-06, 9.9914002e-01, ..., 4.1221973e-04,
        4.1980216e-05, 9.3248409e-06],
       [5.6081103e-07, 1.3757233e-05, 9.7657219e-05, ..., 2.8788078e-05,
        9.3653571e-06, 4.6974142e-06]], dtype=float32)

In [30]:
print(classification_report(np.argmax(y_test,axis=1),np.argmax(preds,axis=1)))

             precision    recall  f1-score   support

          0       0.61      0.60      0.60       486
          1       0.53      0.50      0.52      2889
          2       0.93      0.94      0.93      6128
          3       0.93      0.95      0.94      3336
          4       0.45      0.28      0.35       765
          5       0.53      0.45      0.48      5911
          6       0.75      0.81      0.77     15788
          7       0.60      0.60      0.60      4697

avg / total       0.72      0.73      0.72     40000



In [34]:
model_json = model.to_json()
with open('model_text_classification.json', 'w') as json_file:
    json_file.write(model_json)

model.save_weights('model_text_classification.h5')

In [54]:
preds.shape

(40000, 8)

In [77]:
np.argmax(preds[1])

2

In [92]:
preds[7]

array([4.6864458e-04, 2.6556388e-01, 5.5139052e-04, 2.8467458e-04,
       6.6631177e-04, 1.9748784e-03, 8.2585225e-03, 7.2223175e-01],
      dtype=float32)

In [93]:
#valid_forms = ['am','are','were','was','is','been','being','be']
valid_forms[np.argmax(preds[7])]

'be'

In [94]:
y_test[7]

array([0, 0, 0, 0, 0, 0, 0, 1], dtype=uint8)

In [186]:
# Save Tokenizer i.e. Vocabulary
import pickle
with open('TextClassification.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [187]:
# load tokenizer
import pickle
with open('TextClassification.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

In [210]:
sequences = tokenizer.texts_to_sequences(['Of course. I sorry'])
X_test_testing = pad_sequences(sequences, maxlen=50)
X_test_testing

array([[  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   9, 166,  10, 481]])

In [211]:
preds1 = model.predict(X_test_testing)
preds1

array([[7.3116577e-01, 2.8652309e-03, 3.0756216e-03, 3.0108052e-04,
        3.8060308e-03, 4.6187039e-02, 2.1202692e-01, 5.7228870e-04]],
      dtype=float32)

In [212]:
valid_forms[np.argmax(preds1[0])]

'am'